In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
from IPython.display import display, clear_output

import plotly.express as px #import for 3d interactive plotting


## Main Class

In [52]:
class system():
    
    def __init__(self, name, mass, pos=[0,0,0], gravity=0):
        
        self.bodies = pd.DataFrame(columns=['Position (km)','Mass ()','Velocity (km/s)', 'Gravity (m/s^2)', 'Position Index']) #initializing the body dataframe
        self.positions = np.array([pos]).T #numpy array of shape dimension 2 or 3, number of objects
        
        #Possibility to add diameters/sizes for graphing
#         self.sizes = np.array()
#         np.append(self.sizes, diameter)
        
        self.central_pos = pos #setting the position of the central body
        self.central_name = name #setting the name of the central body
        self.central_mass = mass #setting the mass of the central body
        self.bodies.loc[name] = {'Position (km)': pos, 'Mass ()': mass, 'Velocity (km/s)': [0,0,0], 'Position Index': 0} #adding the parameters of the central body
    
    
    def add_body(self, name, mass, pos, velocity, gravity=0):
        
        index = len(self.bodies) #find the index of the new row

        new_row = {'Position (km)': pos, 'Mass ()': mass, 'Velocity (km/s)': velocity, 'Gravity (m/s^2)': gravity, 'Position Index': index} #create dict of data for new row
        
        pos = np.array([pos]).T #create array of positions of shape dimension 3 by 1
        
        self.bodies.loc[name] = new_row #insert new row in dataframe
        self.positions = np.append(self.positions, pos, axis=1) #append individual position array to total position array
        
#     def add_bodies_from_df(self, names, masses, pos, velocities):
        
#         '''
#         Possible method for adding in n number of bodies.
#         '''
        
#         for i in range(len(names)):
            
#             index = len(self.bodies)
            
#             new_row = {'Position (x,y,z)': pos[i], 'Mass ()': masses[i], 'Velocity ()': velocities[i], 'Position Index': index}
        
#             pos = np.array([pos[i]]).T
        
#             self.bodies.loc[name[i]] = new_row
#             self.positions = np.append(self.positions, pos, axis=1)
    
    def return_positions(self):
    
        return self.positions
    
    def return_bodies(self):
        
        return self.bodies
    
    def update_kinematics(self):
        
        pass
    
    def delete_body(self, body):
        
        index = self.bodies.loc[body,'Position Index'] #pulls body's position index in the array from dataframe
        
        self.positions = np.delete(self.positions, index, 1) #removes the body from the positions array
        self.bodies = self.bodies.drop(body) #removes the body from the dataframe
    
    def timestep(self, time):
        
        #insert update_update kinematics function

        c = np.arange(0,len(self.positions[0].tolist())) #sets a color map so each body has a different color
        
        #initializing x and y values
        x = self.positions[0].tolist() #made a list to make proper length
        y = self.positions[1].tolist()
        z = self.positions[2].tolist()
        
        #create the 3d plot
        scatter3d = px.scatter_3d(x=x, y=y, z=z, color=c, text=self.bodies.index.to_list(), template='plotly_dark')
        
#         scatter3d = scatter3d.update_layout(showgrid=False)

        return scatter3d.show()
    
    def run_sim_static(self, n_iteration, time):

        fig, ax = px.scatter_3d(figsize=(10,5))

        for i in range(0,n_iteration,1):

            self.timestep(1)

            clear_output(wait=True) # Clear output for dynamic display
            display(fig)            # Reset display
            fig.clf()
            time.sleep(0.1)      # Sleep for a fraction of a second to allow animation to catch up
        
        pass
    
    def run_sim_full_output(self, path, prefix, title, n_iteration, axesTF = True):
        '''
        run the simulation with given input parameters and save the output images
        
        path: relative path to were the animation frames will be stored (end with /)
        prefix: generated image filename prefix
        n_iteration: 
        axesTF: conditional for whether to have the axes elements on or off
        
        #ffmpeg -framerate 10 -i Test_3DScatter_s40_cross_%04d.jpeg  Test_3DScatter_s40_3d_fr_cross_Movie.mp4
        '''
        
        #Generate position data for all objects for the length of the simulation, or for a subset of time
            #depending on time length?
            
            
        #Testing
        xPlot, yPlot, zPlot = self.positions[0], self.positions[1], self.positions[2]
        
        #Generate, output, and clear the figure for each frame
        count = 0 #Used to update the image filenames 
        for i in range(0,n_iteration,1):
            fig = plt.figure(figsize = (15,15))
            ax = fig.add_subplot(111, projection='3d')
            
            tempAx = ax.scatter(xPlot, yPlot, zPlot, s = 60, edgecolors = 'black', alpha = 0.8)

            ax.view_init(elev=0, azim=i) #Change view parameters (spin along the azimuth)
            
            ax.set_xlabel('x', fontsize = 15) #Assign label
            ax.set_ylabel('y', fontsize = 15) #Assign label
            ax.set_zlabel('z', fontsize = 15) #Assign label
            
            ax.set_title(title, y=0.95, fontsize=15)
            
            #Set axes state
            ax.axis('on')
            if axesTF == False:
                ax.axis('off')
            
            i_str = str(count)
            suffix = i_str.rjust(4,'0')
            fig.savefig(path + prefix + suffix + '.jpeg')
            fig.clear()
            plt.close(fig)
            count += 1

## Testing

In [51]:
#creating our solar system base (no actual velocities yet)
#pos and vel as of 11/23/21 using JPL Horizons System
solar_system = system('Sun', 1.988E30)
solar_system.add_body(name='Mercury', mass=0.33E24, pos=[-4.407E7,-4.714E7,-2.062E7], velocity=[2.71E1,-2.51E1,-1.62E1], gravity=3.7)
solar_system.add_body(name='Venus', mass=4.87E24, pos=[9.107E7,5.542E7,1.917E7], velocity=[-1.9E1,2.63E1,1.30E1], gravity=8.9)
solar_system.add_body(name='Earth', mass=5.97E24, pos=[7.251E7,1.181E8,5.119E7], velocity=[-2.64E1,1.33E1,5.77E0], gravity=9.8)
solar_system.add_body(name='Moon', mass=0.073E24, pos=[7.244E7,1.185E8,5.137E7], velocity=[-2.74E1,1.31E1,5.76E0], gravity=1.6)
solar_system.add_body(name='Mars', mass=0.642E24, pos=[-1.910E8,-1.294E8,-5.418E7], velocity=[1.524E1,-1.57E1,-7.62E0], gravity=3.7)
solar_system.add_body(name='Jupiter', mass=1898.0E24, pos=[6.803E8,-2.799E8,-1.365E8], velocity=[5.29E0,1.15E1,4.82E0], gravity=23.1)
solar_system.add_body(name='Saturn', mass=568.0E24, pos=[1.020E9,-9.819E8,-4.495E08], velocity=[6.49E0,6.22E0,2.29E0], gravity=9.0)
solar_system.add_body(name='Uranus', mass=86.8E24, pos=[2.170E0,1.844E9,7.768E8], velocity=[-4.66E0,4.28E0,1.94E0], gravity=8.7)
solar_system.add_body(name='Neptune', mass=102.0E24, pos=[4.431E9,-5.428E8,-3.325E8], velocity=[7.363E-1,5.03E0,2.04E0], gravity=11.0)
solar_system.add_body(name='Pluto', mass=0.0146E24, pos=[2.246E9,-4.186E9,-1.983E9], velocity=[5.046E0,1.756E0,-9.75E-1], gravity=0.7)

print(solar_system.return_positions())
solar_system.return_bodies()

[[ 0.000e+00 -4.407e+07  9.107e+07  7.251e+07  7.244e+07 -1.910e+08
   6.803e+08  1.020e+09  2.170e+00  4.431e+09  2.246e+09]
 [ 0.000e+00 -4.714e+07  5.542e+07  1.181e+08  1.185e+08 -1.294e+08
  -2.799e+08 -9.819e+08  1.844e+09 -5.428e+08 -4.186e+09]
 [ 0.000e+00 -2.062e+07  1.917e+07  5.119e+07  5.137e+07 -5.418e+07
  -1.365e+08 -4.495e+08  7.768e+08 -3.325e+08 -1.983e+09]]


,Position (km),Mass (),Velocity (km/s),Gravity (m/s^2),Position Index
Sun,"[0, 0, 0]",1.988000e+30,"[0, 0, 0]",NaN,0
Mercury,"[-44070000.0, -47140000.0, -20620000.0]",3.300000e+23,"[27.1, -25.1, -16.2]",3.7,1
Venus,"[91070000.0, 55420000.0, 19170000.0]",4.870000e+24,"[-19.0, 26.3, 13.0]",8.9,2
Earth,"[72510000.0, 118100000.0, 51190000.0]",5.970000e+24,"[-26.4, 13.3, 5.77]",9.8,3
Moon,"[72440000.0, 118500000.0, 51370000.0]",7.300000e+22,"[-27.4, 13.1, 5.76]",1.6,4
Mars,"[-191000000.0, -129400000.0, -54180000.0]",6.420000e+23,"[15.24, -15.7, -7.62]",3.7,5
Jupiter,"[680300000.0, -279900000.0, -136500000.0]",1.898000e+27,"[5.29, 11.5, 4.82]",23.1,6
Saturn,"[1020000000.0, -981900000.0, -449500000.0]",5.680000e+26,"[6.49, 6.22, 2.29]",9.0,7
Uranus,"[2.17, 1844000000.0, 776800000.0]",8.680000e+25,"[-4.66, 4.28, 1.94]",8.7,8
Neptune,"[4431000000.0, -542800000.0, -332500000.0]",1.020000e+26,"[0.7363, 5.03, 2.04]",11.0,9


In [48]:
#plotting our solar system initial params
#solar_system.timestep(1)

In [49]:
'''
Create a folder nameed AnimationTesting in the same directory as this program to test the program
To create the animation download ffmpeg, navigate to the folder with the animation frames then run the commented line
    at the bottom of this cell in the command line
'''
solar_system.positions
solar_system.run_sim_full_output('AnimationTesting/', 'TestImage', 'TestTitle', 50)

#ffmpeg -framerate 10 -i TestImage%04d.jpeg  TestImage_Movie.mp4

## Secondary Class?

https://plotly.com/python-api-reference/generated/plotly.express.scatter_3d.html
https://www.geeksforgeeks.org/plotly-express-scatter_3d-function-in-python/


https://ssd.jpl.nasa.gov/horizons/app.html#/

In [ ]:
class celestial_body():
    
    def __init__(name, pos, mass, velocity):
        
        self.name = name #string
        self.position = np.array(pos) #list
        self.mass = mass #integer
        self.velocity = np.array(velocity) #list of component velocities
        
    
    